# Red Wine Casus


We gaan aan de slag met een dataset over Portugese rode wijnen. Aan de hand van de gegeven dataset gaan wij een aantal onderzoeksvragen opstellen en beantwoorden.

**Onderzoeksvragen**
1. In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?(verplichte onderzoeksvraag) (Supervised - Regressie)
2. Als we wijnen categoriseren op basis van de chemische samenstelling, zijn er bepaalde categorieën die mannelijke proevers anders beoordelen dan vrouwelijke proevers? (Unsupervised - Clustering)
3. Kun je op basis van bepaalde keywords in de beschrijving een voorspelling doen over hoe hoog de score van deze wijn is? (Supervised - Regression/ Datavisualisatie) 

__We hebben een aantal imports nodig om operaties uit te kunnen voeren binnen het bestand:__

In [ ]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
# nltk.download()

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None)

# 0. Data inlezen

We beginnen met het inlezen van onze dataset "wine". Hieronder tonen we de eerste 5 records van de dataset: 

In [ ]:
wine = pd.read_csv('redwine.csv', delimiter=';')
wine.head()

Het leek ons handig om een beschrijving te hebben over wat de kolommen in de dataset precies inhouden.

__Beschrijving van de kolommen__
<li>Country: Het land van herkomst van de wijn. In deze dataset komt elke wijn uit Portugal, het opslaan van deze kolom is dus redelijk onnodig aangezien het bij elk record hetzelfde is.</li>
<li>Description: Een tekstuele beschrijving over de wijn.</li>
<li>Designation: Vanuit waar de wijn verkocht wordt.</li>
<li>Points: De score van de wijn op een schaal van 1-100.</li>
<li>Price: De prijs van de wijn.</li>
<li>Province: De provincie waar de wijn is gemaakt.</li>
<li>Taster_name: De naam van de persoon die de wijn geproeft heeft.</li>
<li>Title: De naam van de wijn</li>
<li>Variety: Wijnsoort. Ook hier valt het op dat alle wijnen hierbij dezelfde waarde hebben</li>
<li>Winery: De wijnmakerij waar de wijn vandaan komt.</li>

_Chemische samenstelling_
<li>Fixed acidity: vaste zuurtegraad</li>
<li>Volatile acidity: vluchtige zuurtegtraad</li>
<li>Citric acid: citroenzuur</li>
<li>Residual sugar: Natuurlijke druifsuikers die achterblijven in de wijn nadat de alcoholische fermentatie plaats heeft gevonden.</li>
<li>Chlorides: zout</li>
<li>Free sulfur dioxide: ongebonden sulfur dioxide</li>
<li>Total sulfur dioxide: totaal aantal sulfur dioxide in de wijn</li>
<li>Density: dichtheid</li>
<li>pH: pH-waarde</li>
<li>Sulphates: sulfaat</li>
<li>alcohol: alcohol percentage</li>

# 1. Dataverkenning: Wat valt ons op?

We begonnen met het bekijken van de dataset en het documenteren van opvallende waardes. Deze data verkennings fase is belangrijk om later te kunnen beslissen welke gegevens wel of niet bruikbaar zijn om onze onderzoeksvragen te kunnen beantwoorden. 

Zo zie je bijvoorbeeld dat het gemiddelde van het aantal punten van de wijnen 88.8 is. Dit zou dus op een schaal van 1-10 uitkomen op een 8,8. 

In [ ]:
np.mean(wine['points'])

Ook zie je dat het laagst gegeven cijfer een 8,1 is. Redelijk optimistisch. 

In [ ]:
np.min(wine['points'])

Er zijn ook een aantal records waarbij lege waardes voorkomen. We moeten hierbij kijken wat we gaan doen met de lege data en of het invloed heeft op de antwoorden van onze onderzoeksvragen. 

In [ ]:
wine.isnull().sum().sum()

De tabel hieronder toont aan dat alle tasters meer dan één wijnsoort geproeft hebben. Je ziet zelfs dat sommigen er honderden hebben beoordeeld. Roger Voss heeft ruim de meeste wijnen beoordeeld, we moeten bij de onderzoeksvragen goed bedenken wat voor invloed dit heeft op de conclusies die we gaan trekken. 

In [ ]:
nameCount = wine.groupby(['taster_name']).size().reset_index(name='count')
nameCount.sort_values('count', ascending = False)

In [ ]:
wine['taster_name'] = wine['taster_name'].apply(lambda name: name.split(" ")[0])

## 1.1. De chemische samenstelling

De chemische samenstelling is een belangrijk gedeelte van de dataset aangezien we hier bij alle drie de onderzoeksvragen mee aan het werk gaan. Hieronder zien we de chemische samenstelling van de eerste 5 wijnen. We zien dat de waardes nog niet gestandaardiseerd zijn, dit gaan we later doen. 

In [ ]:
chemColNames = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']
chem = wine[chemColNames]
chem.head()

In [ ]:
chem.describe(include = 'all')

# 2. De externe dataset

We willen een externe dataset gebruiken voor het beantwoorden van een van onze onderzoeksvragen. De dataset bestaat uit engelse namen en hun geslacht. Aan de hand van de informatie uit deze dataset willen we voorspellen of de tasters in onze eigen dataset van het mannelijk of vrouwelijke geslacht zijn. 


Hieronder zien we de eerste 10 records van de externe dataset. 

In [ ]:
name = pd.read_csv('names.csv', usecols=['Name', 'Gender'])
name.head(10)

# 3. Data Preparation

## 3.1. Pair Plot

Voordat we een pairplot kunnen maken moeten we de types van een aantal kolommen aanpassen. We willen bijvoorbeeld alcohol plotten en daarvoor moeten we eerst zorgen dat alcohol van het type object naar een nummeriek type geconverteerd wordt. We kunnen terwijl we dit doen ook meteen de waardes van density en citric acid omzetten naar nummerieke datatypes, mogelijk hebben we deze later nog nodig. 

In [ ]:
colErrorPairs = {
    'density'    : [' . '],
    'citric acid': [' - ',' -   '],
    'alcohol'    : ['100.333.333.333.333','11.066.666.666.666.600','956.666.666.666.667','923.333.333.333.333']}

for colName in colErrorPairs:
    for faultyString in colErrorPairs[colName]:
        wine[colName] = wine[colName].replace(faultyString,np.nan)

In [ ]:
wine['alcohol'] = wine['alcohol'].astype(float)

In [ ]:
wine['density'] = wine['density'].astype(float)

In [ ]:
wine['citric acid'] = wine['citric acid'].astype(float)

In [ ]:
wine.dtypes

Nog een terug blik op de chemische samenstelling.<br>
Dit was alleen mogelijk na de data preperation.<br>
Hier zie je bijvoorbeeld het vershil in spreiding, wat opvalt is dat er in suiker maar vooral in sulfur een groot aantal hoge uitschieters zijn.

In [ ]:
wine.boxplot(figsize=(20,10), column=chemColNames)

 Nu dat we de gewenste types hebben omgezet kunnen we ze ook plotten in een pairplot

In [ ]:
sns.set(style="ticks", color_codes=True)
g = sns.pairplot(wine, diag_kind = "hist",
                 height = 4,
                 hue = "taster_name",

                 x_vars=["points", "price", "alcohol"], 
                 y_vars= ["points", "price", "alcohol"])

plt.show()

De pairplot geeft 3 variabelen weer; het alcohol percentage, de prijs en de score van de wijn. We vonden het niet nuttig om de gehele chemische samenstelling ook te verwerken in de pairplot. Op basis van de relatie tussen het alcohol percentage en de gegeven score (points) kunnen we niet perse een directe relatie zien. Wat bijvoorbeeld wel interessant is om op te merken is dat wanneer we kijken naar het aantal points en de prijs van de wijn, is dat de twee duurste wijnen de beste score hebben gekregen. Zo hebben ook de twee meest goedkope wijnen de slechte score gekregen. Er is dus wel een mogelijk verband tussen hoe duur een wijn is en hoeveel punten de wijn heeft gescoort. 

Bij de legende naast de pairplot zien we dat elke proever een eigen kleur heeft gekregen, dit kan omdat er in totaal maar 18 personen zijn die samen het totaal aantal wijnen hebben geproefd. 

## 3.2 Standaardiseren


Stanaardiseren helpt om eerlijker te meten. Je verwijdert de eenheden en zorgt dat alle variabelen ongeveer hetzeflde bereik hebben. 

Alle variabelen die te maken hebben met de chemische samenstelling zouden ongeveer hetzelfde bereik moeten hebben. Als we ervoor zorgen dat deze waardes bijvoorbeeld allemaal tussen de 0 en 1 liggen, kunnen we ze makkelijk met elkaar vergelijken zonder dat er één veel zwaarder weegt dan de ander.


In [ ]:
wineZscore = wine.copy()
for col in chemColNames:
        wineZscore[col] = (wine[col] - wine[col].mean())/wine[col].std(ddof=0)
wineZscore.head()

Hieronder zien we de boxplot met de gestandaardiseerde waardes.
Hier is duidelijk een vershil te zien met de vorige pairplot.

In [ ]:
wineZscore.boxplot(figsize=(20,10), column=chemColNames)